# 词向量与Word2Vec实践

本notebook演示了词向量的使用和训练方法。

## 学习目标
1. 理解词向量的基本概念
2. 学习使用预训练词向量模型
3. 掌握Word2Vec模型的训练方法
4. 了解词向量的常见应用：相似度计算、类比推理等

## 什么是词向量？

词向量（Word Embedding）是将词语映射到实数向量空间的技术。每个词被表示为一个固定维度的向量，语义相近的词在向量空间中距离更近。

**优势：**
- 捕捉词语的语义信息
- 支持向量运算（相似度、类比等）
- 降维表示，节省存储空间

## 1. 环境准备

导入所需的库：
- **pandas**: 数据处理
- **jieba**: 中文分词
- **Word2Vec**: 训练词向量模型
- **KeyedVectors**: 加载和使用预训练词向量

In [41]:
# 导入必要的库
import pandas as pd  # 数据处理
import jieba  # 中文分词
from gensim.models import Word2Vec  # 训练Word2Vec模型
from gensim.models import KeyedVectors  # 加载预训练词向量

## 2. 使用预训练词向量

### 2.1 加载预训练模型

我们使用在微博数据上训练的中文词向量模型（SGNS）：
- **数据来源**: 微博文本
- **训练算法**: Skip-gram with Negative Sampling (SGNS)
- **向量维度**: 300维
- **词表大小**: 约19.5万词

**模型下载**: https://github.com/Embedding/Chinese-Word-Vectors

In [42]:
# 加载预训练的微博词向量模型
# 该模型使用Skip-gram (SGNS)算法在微博数据上训练
# 加载时间较长，请耐心等待（约30-50秒）
model_path = './data/sgns.weibo.word'
model = KeyedVectors.load_word2vec_format(model_path)

### 2.2 词向量基本属性

查看词向量模型的基本信息。

In [43]:
# 1. 查看词向量的维度数目
# vector_size 表示每个词用多少维的向量来表示
# 维度越高，表达能力越强，但计算成本也越高
# 常见维度：50, 100, 200, 300
model.vector_size

300

In [44]:
# 2. 查看词表大小
# index_to_key 存储了模型中所有的词汇
# 词表越大，覆盖的词汇越全面
len(model.index_to_key)

195202

### 2.3 获取词向量

通过索引方式直接获取词的向量表示。

In [45]:
# 3. 查看单个词的词向量
# 通过索引方式获取'地铁'这个词的300维向量表示
# 返回一个numpy数组，包含300个浮点数
model['地铁']

array([ 2.92064e-01, -5.18680e-02, -2.13720e-01,  1.82131e-01,
        2.82900e-03,  4.14104e-01,  1.56440e-01, -1.27940e-02,
       -3.28332e-01, -8.25000e-02, -8.46890e-02, -2.14700e-02,
        1.18650e-01, -4.73659e-01, -1.97850e-02,  1.13939e-01,
        1.82734e-01, -6.46420e-02,  5.60832e-01, -6.65230e-02,
       -1.97960e-01,  1.26039e-01, -3.28720e-01, -3.09730e-02,
       -3.46580e-01, -1.53190e-01, -2.96226e-01, -5.75517e-01,
        1.10684e-01,  8.19220e-02, -1.04721e-01, -1.77477e-01,
       -1.21332e-01,  1.49816e-01,  2.86278e-01, -8.11200e-03,
        6.72540e-02,  6.92220e-02, -3.50973e-01, -5.49500e-02,
       -7.80250e-02, -1.92952e-01, -1.70920e-01, -1.28289e-01,
        1.08204e-01, -7.24913e-01, -1.11735e-01, -6.75000e-03,
        4.38086e-01, -8.75720e-02, -1.41320e-01, -1.91726e-01,
        1.68363e-01, -7.85700e-02, -1.79772e-01, -1.27950e-01,
        3.24675e-01,  2.70616e-01,  1.96330e-02, -3.09431e-01,
       -4.02670e-02,  5.80160e-02, -1.06603e-01,  2.480

### 2.4 词语相似度计算

**相似度计算方法**: 余弦相似度（Cosine Similarity）

$$
\text{similarity}(\vec{A}, \vec{B}) = \frac{\vec{A} \cdot \vec{B}}{||\vec{A}|| \times ||\vec{B}||}
$$

**相似度范围**: [-1, 1]
- 1: 完全相同
- 0: 无关
- -1: 完全相反

In [46]:
# 4. 计算两个词之间的余弦相似度
# 相似度范围是[-1, 1]，值越大表示两个词语义越相似
# '公交'和'地铁'都是交通工具，语义相近，相似度应该较高
model.similarity("公交","地铁")

np.float32(0.65458214)

### 2.5 词向量的类比推理

词向量最有趣的特性之一是支持类比推理（Word Analogy）。

**经典例子**: 
- 国王 - 男人 + 女人 ≈ 女王
- 北京 - 中国 + 日本 ≈ 东京

**API说明**: `most_similar(positive=[...], negative=[...])`
- **positive**: 正向词列表（相加）
- **negative**: 负向词列表（相减）
- **返回**: 最相似的词及其相似度

In [47]:
# 5. 词向量的类比推理
# positive: 正向词，negative: 负向词
# 计算公式: vector('男人') + vector('女孩') - vector('男孩')
# 结果应该接近'女人'，验证了 男人-男孩 ≈ 女人-女孩 的语义关系
# 这展示了词向量能够捕捉性别、年龄等语义维度
model.most_similar(positive=['男人','女孩'],negative=['男孩'])

[('女人', 0.6578881740570068),
 ('女孩子', 0.515068531036377),
 ('女生', 0.45194485783576965),
 ('女人真', 0.4420627951622009),
 ('女人们', 0.43698593974113464),
 ('女人爱', 0.435453325510025),
 ('寡言少语', 0.42479243874549866),
 ('男孩子', 0.4217747151851654),
 ('看女人', 0.4194916784763336),
 ('笨女人', 0.41820037364959717)]

## 3. 训练自己的Word2Vec模型

### 3.1 数据准备

使用在线购物评论数据集训练词向量模型。

In [54]:
# 加载在线购物评论数据集
# 包含10个商品类别的评论文本
# dropna()删除缺失值，确保数据质量
df= pd.read_csv('./data/online_shopping_10_cats.csv',encoding='utf-8').dropna()

### 3.2 文本分词

Word2Vec要求输入为分词后的句子列表。

In [59]:
# 对评论文本进行分词处理
# 1. 使用jieba.lcut()对每条评论进行分词
# 2. 过滤掉空白字符，保留有效的词语
# 3. 生成适合Word2Vec训练的句子列表
# 格式: [[词1, 词2, ...], [词1, 词2, ...], ...]
sentences = [[token for token in jieba.lcut(sentence) if token.strip() != ''] for sentence in df['review']]

In [60]:
# 查看前3条分词结果
# 每条评论被分成一个词列表
sentences[0:3]

[['做', '父母', '一定', ...],
 ['作者', '真有', '英国人', ...],
 ['作者', '长篇大论', '借用', ...]]

### 3.3 训练Word2Vec模型

**Word2Vec的两种算法:**

1. **CBOW (Continuous Bag of Words)**
   - 根据上下文预测目标词
   - 速度快，适合大规模数据

2. **Skip-gram**
   - 根据目标词预测上下文
   - 对低频词效果更好
   - 本例使用此算法

**关键参数说明:**

| 参数 | 说明 | 推荐值 |
|------|------|--------|
| vector_size | 词向量维度 | 50-300 |
| window | 上下文窗口大小 | 5-10 |
| min_count | 最小词频 | 2-5 |
| sg | 算法选择 | 0=CBOW, 1=Skip-gram |
| workers | 并行线程数 | CPU核心数 |

In [61]:
# 训练Word2Vec模型
# sentences: 分词后的句子列表
# vector_size=100: 词向量维度为100（比预训练模型的300维小，训练更快）
# window=5: 上下文窗口大小为5，即考虑前后各5个词
# min_count=2: 过滤出现次数少于2次的词（去除罕见词，提高训练效率）
# sg=1: 使用Skip-gram算法(sg=0为CBOW算法)
# workers=4: 使用4个线程并行训练（加速训练过程）
w2v_model=Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=1,
    workers=4
)

### 3.4 保存和加载模型

**保存格式说明:**
- **文本格式**: 易读，兼容性好，但文件较大
- **二进制格式**: 文件小，加载快，但不可读

In [62]:
# 保存训练好的词向量到文件
# wv属性包含了词向量数据（Word2Vec对象还包含训练状态等其他信息）
# save_word2vec_format() 保存为标准的word2vec文本格式
# 格式：第一行是词表大小和向量维度，后续每行是一个词及其向量
# 方便后续加载使用，也可以被其他工具读取
w2v_model.wv.save_word2vec_format("./data/word2vec.txt")

### 3.5 验证训练结果

加载保存的模型并验证词向量。

In [63]:
# 验证：加载保存的词向量文件
# 查看训练模型中'地铁'的词向量表示
# 注意：这是我们自己训练的100维向量，与预训练模型的300维不同
# 向量值也会不同，因为训练数据和参数都不一样
KeyedVectors.load_word2vec_format('./data/word2vec.txt')['地铁']

array([ 0.53869164,  0.4346466 , -0.7772162 , -0.03371358,  0.46554598,
       -0.7254186 ,  0.26502004,  0.50014   , -0.31351703,  0.12573367,
       -0.039761  , -0.75900096,  0.19005717, -0.32584673,  0.23872773,
       -0.12806633,  0.5544915 , -0.98756284,  0.01115693, -0.08069381,
        0.43849576,  0.03989036,  0.14404255, -0.28249308, -0.6250146 ,
        0.137326  ,  0.18429303,  0.00149757,  0.23669262, -0.06547165,
        0.18620954, -0.13052796,  0.18064444, -0.20118159,  0.53292686,
       -0.08744698, -0.04238345,  0.22264194, -0.06850787, -0.39956895,
       -0.56638384,  0.2235286 ,  0.10352888,  0.16257638,  0.80463356,
       -0.23929346,  0.14425576,  0.4596448 ,  0.3885115 ,  0.18302162,
       -0.31518784,  0.08163848, -0.28451753,  0.01862862,  0.5904339 ,
       -0.17792872,  0.5003327 ,  0.5777704 , -0.19197373,  0.2928307 ,
        0.15448211,  0.28496897,  0.08314283, -0.02761365,  0.4087935 ,
        0.18068857,  0.14396985,  0.8015779 , -0.12802924,  0.24

## 4. 总结与最佳实践

### 4.1 KeyedVectors常用API

| 方法 | 功能 | 示例 |
|------|------|------|
| `model[word]` | 获取词向量 | `model['地铁']` |
| `similarity(w1, w2)` | 计算相似度 | `model.similarity('公交', '地铁')` |
| `most_similar(positive, negative)` | 类比推理 | `model.most_similar(positive=['男人','女孩'], negative=['男孩'])` |
| `doesnt_match(words)` | 找出不匹配的词 | `model.doesnt_match(['早餐','午餐','晚餐','电脑'])` |
| `distance(w1, w2)` | 计算距离 | `model.distance('公交', '地铁')` |
| `most_similar_topn(word, topn)` | 找最相似的N个词 | `model.most_similar('地铁', topn=10)` |

### 4.2 Word2Vec训练建议

**数据要求:**
- 语料规模：至少几百万词
- 数据质量：去除噪声，保持语义完整性
- 分词质量：影响最终效果

**参数调优:**
- **小语料** (< 1M词): vector_size=50-100, window=5, min_count=1
- **中语料** (1M-100M词): vector_size=100-200, window=5-10, min_count=5
- **大语料** (> 100M词): vector_size=200-300, window=10, min_count=10

**选择建议:**
- **使用预训练模型**: 适合大部分通用场景
- **自己训练模型**: 适合特定领域（如医疗、金融）
- **微调预训练模型**: 在预训练基础上继续训练

### 4.3 词向量的应用

1. **文本分类**: 将词向量平均作为文本特征
2. **命名实体识别**: 作为神经网络的输入层
3. **机器翻译**: 编码源语言语义
4. **问答系统**: 计算问题和答案的语义相似度
5. **推荐系统**: 计算物品描述的相似度